# **MultilatSensorNet Performance Analysis**

# **Setup**

In [ ]:
!pip install plotly

In [48]:
import plotly.graph_objects as go
from scipy.interpolate import interp1d
import pandas as pd
import numpy as np

# **Trajectory Analysis**

---
**DataFrames Creation**

In [44]:
tracked_lin_df = pd.read_csv("../data/run_linear.csv", sep=';')
true_lin_df = pd.read_csv("../data/target_linear_trajectory.csv", sep=';')
tracked_circ_df = pd.read_csv("../data/run_circular.csv", sep=';')
true_circ_df = pd.read_csv("../data/target_circular_trajectory.csv", sep=';')
sensors_pos_df = pd.read_csv("../data/sensors_pos.csv", sep=';')

---
**Target and Tracked Trajectories 3D Plot**

In [45]:
def plot_trajectories(true_df: pd.DataFrame, tracked_df: pd.DataFrame, sensors_pos_df: pd.DataFrame) -> None:
    """Plots 3D trajectories for true and tracked positions along with sensor positions.

    This function visualizes the true trajectory, tracked trajectory, and sensor positions in a 3D space
    using Plotly. It is useful for comparing the actual and estimated paths and observing the distribution of sensors.

    Args:
        true_df: A pandas DataFrame containing true trajectory positions with columns 'X', 'Y', and 'Z'.
        tracked_df: A pandas DataFrame containing tracked trajectory positions with columns 'X', 'Y', and 'Z'.
        sensors_pos_df: A pandas DataFrame containing sensor positions with columns 'X', 'Y', and 'Z'.
    """
    # Creates the figure object
    fig = go.Figure()

    # Adds tracker trajectory in red
    fig.add_trace(go.Scatter3d(
        x=tracked_df['X'], y=tracked_df['Y'], z=tracked_df['Z'],
        mode='lines',
        name='Tracker Trajectory',
        line=dict(color='red')
    ))

    # Plots sensors in the 3D space with green markers
    fig.add_trace(go.Scatter3d(
        x=sensors_pos_df['X'], y=sensors_pos_df['Y'], z=sensors_pos_df['Z'],
        mode='markers',
        name='Sensors',
        marker=dict(color='green', size=5, symbol='circle')
    ))

    # Adds true trajectory in blue
    fig.add_trace(go.Scatter3d(
        x=true_df['X'], y=true_df['Y'], z=true_df['Z'],
        mode='lines',
        name='True Trajectory',
        line=dict(color='blue')
    ))

    # Updates the layout with labels and title
    fig.update_layout(
        title='Target and Tracked 3D Trajectories',
        scene=dict(
            xaxis_title='X axis',
            yaxis_title='Y axis',
            zaxis_title='Z axis',
            xaxis=dict(range=[0, 4.5]),
            yaxis=dict(range=[0, 4.5]),
            zaxis=dict(range=[0, 2.75])
        )
    )

    # Shows the plot
    fig.show()

In [46]:
plot_trajectories(true_lin_df, tracked_lin_df, sensors_pos_df)
plot_trajectories(true_circ_df, tracked_circ_df, sensors_pos_df)

---
**Residual Analysis**

In [49]:
def compute_residuals(true_df: pd.DataFrame, tracked_df: pd.DataFrame) -> pd.DataFrame:
    """Computes residuals between true and tracked positions in 3D space.

    This function calculates the Euclidean distance between corresponding points in the true and tracked
    trajectories after resampling the tracked positions to match the number of points in the true data.

    The tracked trajectory is interpolated to match the number of points in the true trajectory.
    Residuals are computed as Euclidean distances in 3D space.

    Args:
        true_df: A pandas DataFrame containing the true trajectory positions with columns 'X', 'Y', and 'Z'.
        tracked_df: A pandas DataFrame containing the tracked trajectory positions with columns 'X', 'Y', and 'Z'.

    Returns:
        A pandas DataFrame containing the computed Euclidean residuals labeled as 'Residuals'.
    """
    # Interpolate the tracked data to match target data points
    interp_func = interp1d(np.linspace(0, 1, len(tracked_df)), tracked_df, axis=0, fill_value="extrapolate")

    tracked_resampled = interp_func(np.linspace(0, 1, len(true_df)))

    # Compute residuals
    residuals = np.linalg.norm(true_df.values - tracked_resampled, axis=1)
    residual_df = pd.DataFrame(residuals, columns=['Residuals'])

    return residual_df

residual_df = compute_residuals(true_lin_df, tracked_lin_df)
residual_df.describe()

,Residuals
count,100.000000
mean,0.005856
std,0.002407
min,0.001431
25%,0.003961
50%,0.006004
75%,0.007594
max,0.010854


In [50]:
residual_df = compute_residuals(true_circ_df, tracked_circ_df)
residual_df.describe()

,Residuals
count,100.000000
mean,0.029816
std,0.014839
min,0.003162
25%,0.020727
50%,0.022752
75%,0.037021
max,0.073693
